# **1. 서울 열린데이터 광장**
[서울 열린데이터 광장](https://data.seoul.go.kr/)(Seoul Open Data Plaza)은 서울시에서 운영하는 공공데이터 개방 플랫폼입니다. 시민, 연구자, 기업 등이 서울시에서 생성한 다양한 공공데이터를 자유롭게 활용할 수 있도록 제공하고 있습니다. 이를 통해 데이터 기반의 창의적인 아이디어와 혁신을 촉진하며, 시민들의 정보 접근성을 높이고 공공서비스를 개선하는 데 기여하고 있습니다.

# **2. 서울시 공공자전거 실시간 대여정보**
6f727a6c77746c7337344a4f704d4c


In [1]:
import requests
import folium
import json
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [3]:
base_url = 'http://openapi.seoul.go.kr:8088/6f727a6c77746c7337344a4f704d4c/json/bikeList/1/5/'
request = requests.get(base_url)
print(request)

<Response [200]>


In [4]:
json_data = request.json()
json_data

{'rentBikeStatus': {'list_total_count': 5,
  'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다.'},
  'row': [{'rackTotCnt': '15',
    'stationName': '102. 망원역 1번출구 앞',
    'parkingBikeTotCnt': '0',
    'shared': '0',
    'stationLatitude': '37.55564880',
    'stationLongitude': '126.91062927',
    'stationId': 'ST-4'},
   {'rackTotCnt': '14',
    'stationName': '103. 망원역 2번출구 앞',
    'parkingBikeTotCnt': '20',
    'shared': '143',
    'stationLatitude': '37.55495071',
    'stationLongitude': '126.91083527',
    'stationId': 'ST-5'},
   {'rackTotCnt': '13',
    'stationName': '104. 합정역 1번출구 앞',
    'parkingBikeTotCnt': '3',
    'shared': '23',
    'stationLatitude': '37.55073929',
    'stationLongitude': '126.91508484',
    'stationId': 'ST-6'},
   {'rackTotCnt': '5',
    'stationName': '105. 합정역 5번출구 앞',
    'parkingBikeTotCnt': '0',
    'shared': '0',
    'stationLatitude': '37.55000687',
    'stationLongitude': '126.91482544',
    'stationId': 'ST-7'},
   {'rackTotCnt': '12',
   

In [6]:
json_data['rentBikeStatus']['RESULT']['CODE']

'INFO-000'

In [8]:
json_data.get("rentBikeStatus", {}).get("RESULT", {}).get("CODE", "")

'INFO-000'

In [11]:
def fetch_bike_data():
  base_url = "http://openapi.seoul.go.kr:8088/6f727a6c77746c7337344a4f704d4c/json/bikeList/"
  start = 1
  end = 1000
  step = 1000
  data_frames = []

  while True:
    url = f'{base_url}{start}/{end}/'
    request = requests.get(url)
    if request.status_code != 200:
      print(f'status code: {request.status_code}')
      break

    json_data = request.json()
    try:
      rent_bike_status = json_data['rentBikeStatus']
      result_code = rent_bike_status['RESULT']['CODE']
    except KeyError:
      print('결과 코드에 대한 JSON 오류')
      break

    if result_code == 'INFO-200':
      print("더이상 데이터가 없음")
      break
    elif result_code == 'INFO-000':
      print(f'시작: {start} 끝: {end}')
      try:
        bike_data = rent_bike_status['row']
        if bike_data:
          df = pd.DataFrame(bike_data)
          data_frames.append(df)
      except KeyError:
        print("데이터를 찾을 수 없음")
    else:
      print(f'result code: {result_code}')
      break

    start += step
    end += step

  if data_frames:
    final_df = pd.concat(data_frames, ignore_index=True)
    return final_df
  else:
    return pd.DataFrame()

In [12]:
bike_data_df = fetch_bike_data()

시작: 1 끝: 1000
시작: 1001 끝: 2000
시작: 2001 끝: 3000
결과 코드에 대한 JSON 오류


In [13]:
bike_data_df

,rackTotCnt,stationName,parkingBikeTotCnt,shared,stationLatitude,stationLongitude,stationId
0,15,102. 망원역 1번출구 앞,0,0,37.55564880,126.91062927,ST-4
1,14,103. 망원역 2번출구 앞,18,129,37.55495071,126.91083527,ST-5
2,13,104. 합정역 1번출구 앞,4,31,37.55073929,126.91508484,ST-6
3,5,105. 합정역 5번출구 앞,0,0,37.55000687,126.91482544,ST-7
4,12,106. 합정역 7번출구 앞,3,25,37.54864502,126.91282654,ST-8
...,...,...,...,...,...,...,...
2753,8,6185.가양나들목,12,150,37.57341003,126.84345245,ST-3418
2754,12,6187.마곡119안전센터 맞은편,16,133,37.55534744,126.82072449,ST-3415
2755,10,6188.금호아파트,1,10,37.55619049,126.86463928,ST-3419
2756,11,6189.데시앙플렉스 지식산업센터,24,218,37.56448364,126.84830475,ST-3424


```
rackTotCnt  거치대개수
parkingBikeTotCnt   자전거주차총건수
shared  거치율
stationLatitude 위도
stationLongitude    경도
stationId   대여소ID
stationName 대여소이름
```

In [15]:
bike_data_df['stationLatitude'] = bike_data_df['stationLatitude'].astype(float)
bike_data_df['stationLongitude'] = bike_data_df['stationLongitude'].astype(float)

In [17]:
bike_map = folium.Map(location=[bike_data_df['stationLatitude'].mean(),
                                bike_data_df['stationLongitude'].mean()], zoom_start=12)

for index, data in bike_data_df.iterrows():
  popup_str = '{}자전거 주차 총 건수: {}대'.format(data['stationName'], data['parkingBikeTotCnt'])
  popup = folium.Popup(popup_str, max_width=600)
  folium.Marker(location=[data['stationLatitude'], data['stationLongitude']], popup=popup).add_to(bike_map)

bike_map